In [3]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np

The following is a class that storing the distance and the prediction

In [4]:
class distancePredictedResult:
    def __init__(self, distance, predictedResult):
        self.distance = distance
        self.predictedResult = predictedResult

The following is to read the dataset from a csv file.

In [5]:
#load the dataset
def loadDataset(filename, header):
    df = pd.read_csv(filename)
    #map the result 'M' and 'B' to 1 and 0 respectively
    df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})
    #convert all the values from string to float
    df = df.astype(float)
    #add header to the dataframe
    df.columns = header
    #rearrange the column diagnosis: move it to the end
    df = df[[c for c in df if c not in ['diagnosis']] + ['diagnosis']]
    #drop the 'id' column
    df = df.drop('id', 1)
    #data normalization
    normalized_df = (df - df.min()) / (df.max() - df.min())
    return normalized_df

The followings are different way to split the data

In [6]:
#randomly split the data (82%: training data, 28%: testing data)
def splitDataset_random(df):
    train, test = train_test_split(df, test_size=0.28)
    return train, test

#the first 469 rows are training set and the remaining 100 rows are testing set
def splitDataset_1(df):
    train, test = df[:469], df[469:]
    return train, test

The following is to find out the euclideanDistance of two instances i.e. two datas

In [7]:
#find out the euclideanDistance of two instances
import math
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow(float(instance1[x])-float(instance2[x]), 2)
    return math.sqrt(distance)

The followings are to get a sorted list storing all the neighbors according to the distance.
The first one is just simply get the sorted list.
The second one is the get the sorted neighbors having distance smaller than the threshold.

In [8]:
#get the sorted neighbors
def getNeighbors(trainingSet, predictData):
    neighbors = []
    length = len(trainingSet.iloc[0])-1
    for index, row in trainingSet.iterrows():
        tempDistance = euclideanDistance(predictData, row, length)
        neighbors.append(distancePredictedResult(tempDistance, row[length]))
    neighbors.sort(key=lambda x: x.distance)
    return neighbors

#get the sorted neighbors with threshold
def getNeighbors_threshold(trainingSet, predictData):
    neighbors = []
    neighbors_distance_only = []
    length = len(trainingSet.iloc[0])-1
    for index, row in trainingSet.iterrows():
        tempDistance = euclideanDistance(predictData, row, length)
        neighbors.append(distancePredictedResult(tempDistance, row[length]))
        neighbors_distance_only.append(tempDistance)
    neighbors.sort(key=lambda x: x.distance)
    threshold = np.mean(neighbors_distance_only) + np.std(neighbors_distance_only)
    neighbors_threshold = []
    for i in range(len(neighbors)):
        if neighbors[i].distance < threshold:
            neighbors_threshold.append(neighbors[i])
    return neighbors_threshold

The followings are two different prediction. One is without adding weights to the votes according to distance and the other one is with adding weights to the votes according to distance

In [9]:
#get the prediction without adding weights to the votes according to distance
import operator
def getPrediction(dataPoints, k):
    neighbors = []
    for i in range(k):
        neighbors.append(dataPoints[i])
    votes = {}
    for i in range(len(neighbors)):
        classResult = neighbors[i].predictedResult
        if classResult not in votes:
            votes[classResult] = 1
        else:
            votes[classResult] += 1
    sortedKeys = sorted(votes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedKeys[0][0]

#get the prediction with adding weights to the votes according to distance
def getPredictionWithWeight(dataPoints, k):
    neighbors = []
    for i in range(k):
        neighbors.append(dataPoints[i])
    votes = {}
    for i in range(len(neighbors)):
        classResult = neighbors[i].predictedResult
        if classResult not in votes:
            votes[classResult] = 1/neighbors[i].distance
        else:
            votes[classResult] += 1/neighbors[i].distance
    sortedKeys = sorted(votes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedKeys[0][0]

#get the prediction and if there is a tie, reduce k
def getPrediction_dealing_tie(dataPoints, k):
    found_result = False
    while not found_result:
        neighbors = []
        for i in range(k):
            neighbors.append(dataPoints[i])
        votes = {}
        for i in range(len(neighbors)):
            classResult = neighbors[i].predictedResult
            if classResult not in votes:
                votes[classResult] = 1
            else:
                votes[classResult] += 1
        sortedKeys = sorted(votes.items(), key=operator.itemgetter(1), reverse=True)
        if len(sortedKeys) > 1:
            if sortedKeys[0][1] == sortedKeys[1][1]:
                k -= 1
            else:
                return sortedKeys[0][0]
        else:
            return sortedKeys[0][0]

The following is to compare the predictions with the ground truth in order to get the accuracy

In [10]:
#get the accuracy by comparing the predictions with the ground truth
def getAccuracy(testingSet, predictions, columnToBePredict):
    votes = {'true':0, 'false':0}
    for index, row in testingSet.iterrows():
        if row[columnToBePredict] == predictions[index]:
            votes['true'] += 1
        #print('prediction: ', predictions[index], '     result: ', row[columnToBePredict])
        
    return (votes['true']/float(len(testingSet)))*100

The following are different experiments stated in the assignment sheet

In [23]:
#this is the normal way to implement k-nn
def normalAgorithm():
    filename = 'wisc_bc_data.csv'
    header = ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'points_mean', 'symmetry_mean', 'dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'points_se', 'symmetry_se', 'dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'points_worst', 'symmetry_worst', 'dimension_worst']
    columnToBePredict = 'diagnosis'
    normalized_df = loadDataset(filename, header)
    trainingSet, testingSet = splitDataset_1(normalized_df)
    predictions = {}
    k = int(math.sqrt(len(trainingSet)))

    for index, row in testingSet.iterrows():
        tempNeighbors = getNeighbors(trainingSet, row)
        predictions[index] = getPrediction(tempNeighbors, k)
    
    print('Accuracy when using the normal way to implement the algorithm: ', getAccuracy(testingSet, predictions, columnToBePredict), '%')

#adding weights to the votes according to distance
def normalAgorithm_adding_distance_weight():
    filename = 'wisc_bc_data.csv'
    header = ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'points_mean', 'symmetry_mean', 'dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'points_se', 'symmetry_se', 'dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'points_worst', 'symmetry_worst', 'dimension_worst']
    columnToBePredict = 'diagnosis'
    normalized_df = loadDataset(filename, header)
    trainingSet, testingSet = splitDataset_1(normalized_df)
    predictions = {}
    k = int(math.sqrt(len(trainingSet)))

    for index, row in testingSet.iterrows():
        tempNeighbors = getNeighbors(trainingSet, row)
        predictions[index] = getPredictionWithWeight(tempNeighbors, k)
    
    print('Accuracy when adding weight according to the distance: ', getAccuracy(testingSet, predictions, columnToBePredict), '%')
    
#try different k values
#the following try the square root of the number of data of the training set, half of the training set, one third of the training set and one forth of the training set
def different_k_Agorithm():
    filename = 'wisc_bc_data.csv'
    header = ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'points_mean', 'symmetry_mean', 'dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'points_se', 'symmetry_se', 'dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'points_worst', 'symmetry_worst', 'dimension_worst']
    columnToBePredict = 'diagnosis'
    normalized_df = loadDataset(filename, header)
    trainingSet, testingSet = splitDataset_1(normalized_df)
    predictions = {}
    #try different k values
    k = [int(math.sqrt(len(trainingSet))), int(len(trainingSet)/2), int(len(trainingSet)/3), int(len(trainingSet)/4)]
    
    for i in range(len(k)):
        for index, row in testingSet.iterrows():
            tempNeighbors = getNeighbors(trainingSet, row)
            predictions[index] = getPrediction(tempNeighbors, k[i])
        print('Accuracy when trying different k values (',k[i], '): ', getAccuracy(testingSet, predictions, columnToBePredict), '%')
        
#if there is a tie, reduce k until there is a clearer winner
def algorithm_dealing_tie():
    filename = 'wisc_bc_data.csv'
    header = ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'points_mean', 'symmetry_mean', 'dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'points_se', 'symmetry_se', 'dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'points_worst', 'symmetry_worst', 'dimension_worst']
    columnToBePredict = 'diagnosis'
    normalized_df = loadDataset(filename, header)
    trainingSet, testingSet = splitDataset_1(normalized_df)
    predictions = {}
    k = int(math.sqrt(len(trainingSet)))

    for index, row in testingSet.iterrows():
        tempNeighbors = getNeighbors(trainingSet, row)
        predictions[index] = getPrediction_dealing_tie(tempNeighbors, k)
    
    print('Accuracy when using the normal way to implement the algorithm (handle tie): ', getAccuracy(testingSet, predictions, columnToBePredict), '%')
    
#split the data randomly using the normal algorithm
def algorithm_random_split_data():
    filename = 'wisc_bc_data.csv'
    header = ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'points_mean', 'symmetry_mean', 'dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'points_se', 'symmetry_se', 'dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'points_worst', 'symmetry_worst', 'dimension_worst']
    columnToBePredict = 'diagnosis'
    normalized_df = loadDataset(filename, header)
    trainingSet, testingSet = splitDataset_random(normalized_df)
    predictions = {}
    k = int(math.sqrt(len(trainingSet)))

    for index, row in testingSet.iterrows():
        tempNeighbors = getNeighbors(trainingSet, row)
        predictions[index] = getPredictionWithWeight(tempNeighbors, k)
    
    print('Accuracy when spliting the data randomly: ', getAccuracy(testingSet, predictions, columnToBePredict), '%')

In [24]:
def main():
    normalAgorithm()
    normalAgorithm_adding_distance_weight()
    different_k_Agorithm()
    algorithm_dealing_tie()
    algorithm_random_split_data() 

In [25]:
if __name__ == "__main__":
    main()

Accuracy when using the normal way to implement the algorithm:  98.0 %
Accuracy when adding weight according to the distance:  98.0 %
Accuracy when trying different k values ( 21 ):  98.0 %
Accuracy when trying different k values ( 234 ):  89.0 %
Accuracy when trying different k values ( 156 ):  91.0 %
Accuracy when trying different k values ( 117 ):  92.0 %
Accuracy when using the normal way to implement the algorithm (handle tie):  98.0 %
Accuracy when spliting the data randomly:  97.5 %


From different experiments, I found out that by spliting data randomly and adding weight according to the distance can sometimes get an accuracy that is higher than 98% but it is not a must as sometimes it may be lower than 98%.